### This notebook is to generate df of Positions, that is resulted in our strategy

In [1]:
import pandas as pd
from math import sqrt
import math

import requests
import json
import numpy as np

import UNI_v3_funcs as liq_amounts
import load_data

### Query Functions

In [2]:
def get_cprice_tick_at_mint(token0, token1, feeTier, mint_time, decimal0, decimal1):
    
    # get token_ids
    token0_id = load_data.get_token_id(token0)
    token1_id = load_data.get_token_id(token1)
    pool_id = load_data.get_pool_id(token0_id, token1_id, feeTier)
        
    # query price at that specific time
    query_ = """
    {{
      poolHourDatas(first:1,
      where:{{ pool: "{}",
      periodStartUnix: {} }}
      ){{
        tick
      }}
    }}""".format(pool_id, mint_time)
    
    query_result_ = load_data.run_query(query_)
    json_data_ = json.loads(query_result_.text)
    
    ctick = int(json_data_['data']['poolHourDatas'][0]['tick'])
    cprice = tick_to_price(ctick,decimal0, decimal1)
    
    return cprice, ctick

def get_decimals(token_symbol):
    query_ = """
    {{
      tokens(first:10,
      where:{{ symbol: "{}"}}
      ){{
        decimals
      }}
    }}""".format(token_symbol)
    
    query_result_ = load_data.run_query(query_)
    json_data_ = json.loads(query_result_.text)
    decimal_ = float(json_data_['data']['tokens'][0]['decimals'])
    
    return decimal_

### Positions Genrations

In [9]:
def get_positions_fixed_strategy(mint_time, burn_time, upper_, lower_, decimal0_, decimal1_):
    
    df_positions = pd.DataFrame()
    df_positions['periodStartUnix'] = list(range(mint_time, burn_time+1, 3600))
    df_positions['upper'] = upper_
    df_positions['lower'] = lower_
    df_positions['upper_tick'] = price_to_tick(upper_, decimal0_, decimal1_)
    df_positions['lower_tick'] = price_to_tick(lower_, decimal0_, decimal1_)
    
    return df_positions

### Calculations Functions

In [3]:
def price_to_tick(price_, decimal0_, decimal1_):
    # convert price (token1 by token0) to tick
    tick_ = math.floor( math.log(price_*10**(decimal1_-decimal0_), 1.0001) )
    return tick_

def tick_to_price(tick_,decimal0_, decimal1_):
    price_ = (1.0001**tick_/10**(decimal1_-decimal0_))
    return price_

In [19]:
def check_inRange(ctick, lower_tick, upper_tick):

    if int(ctick) > int(upper_tick):
        inRange = False
    elif int(ctick) < int(lower_tick):
        inRange = False
    else: 
        inRange = True
    
    return inRange

In [35]:
# Calculate Fees
# TODO: Clean this up, take out features that we dont need to calculate fees
def get_fees(lower_tick_, upper_tick_, ctick_,  L_Pool_, swap_Volume_, amount0_p, amount1_p, amt0_, amt1_, decimal0_, decimal1_, feeTier, mint_tick):
    # (lower_tick_, upper_tick_) from df_positions
    # (ctick, L_Pool, swap_volume, amount0_p, amount1_p) from df_merged
    # (amt0, amt1) - this changes based on swaps, but doesnt matter, bacause liquidity always constant
    # (decimal0, decimal1, feetier, mint_tick) - fixeds
    
    mint_liquidity = liq_amounts.get_liquidity(mint_tick, lower_tick_,upper_tick_, amt0_, amt1_, decimal0_, decimal1_)
    L_you_ = mint_liquidity # always constant, according to JNP
    
    inRange_ = check_inRange(ctick_, lower_tick_,upper_tick_)
    
    if inRange_:
        pool_share_ = (L_you_/L_Pool_)
        pool_fee_rate_ = float(feeTier) / 10000
        fees_0 = pool_share_ * amount0_p * pool_fee_rate_ / 100
        fees_1 = pool_share_ * amount1_p * pool_fee_rate_ / 100
    else:
        fees_0 = 0
        fees_1 = 0
        pool_share_ = 0
    
    return fees_0, fees_1, pool_share_, L_you_

In [34]:
def calculate_pnl_from_positions(df_merged, df_positions):
    # TODO: re-calculate amount0 and amount1 to account for every swaps, not just deltas of that hour
    # use amount0_new and amount1_new instead of amountUSD, and get total accured fees in tokens
    # Swap_amountIn > so the fees is get from the (+) positive tokens
    # Therefore, fees you get is in 
    # Convert those tokens into value (convert using TODAY's value or mints's value?)
    
    merge_feat = ['periodStartUnix', 'tick','liquidity', 'amountUSD', 'amount0_p', 'amount1_p']

    df_positions_fees = df_positions.merge(df_merged[merge_feat], how='left', on='periodStartUnix')
    df_positions_fees['liquidity'] = df_positions_fees['liquidity'].astype(float)

    df_positions_fees = df_positions_fees.reset_index() # cleaning it just in case
    for index, row in df_positions_fees.iterrows():
        row = row.copy()
        fees_0, fees_1, pool_share, L_you = get_fees(row['lower_tick'],row['upper_tick'],row['tick'],
                        float(row['liquidity']),row['amountUSD'],
                        row['amount0_p'], row['amount1_p'],
                        MINT_AMT0, MINT_AMT1, DECIMAL0, DECIMAL1, FEETIER, MINT_TICK)
       
        df_positions_fees.loc[index, 'fees_0'] = fees_0
        df_positions_fees.loc[index, 'fees_1'] = fees_1
        df_positions_fees.loc[index, 'pool_share'] = pool_share
        df_positions_fees.loc[index, 'L_you'] = L_you
        
    return df_positions_fees

### Main

In [130]:
# STEP 0:  PROVIDE BY USERS
UPPER = 1/1844.6164 # 1/1844.6164  # token1 per token0 (in terms of price)
LOWER = 1/2858.3641 # 1/2858.3641  # 1/2376.6075 
FEETIER = '3000' # 0.3 = 3000/10000
TOKEN0 = 'USDC'
TOKEN1 = 'WETH'
# needs to start with the hour (unixTime % 3600 = 0)
MINT_TIME = 1622304000 # 4PM May 29 (UTC/GMT)
BURN_TIME = 1627866000 # 1AM Aug 02 (UTC/GMT)


# Get required constants
DECIMAL0 = get_decimals(TOKEN0)
DECIMAL1 = get_decimals(TOKEN1)
print(DECIMAL0, DECIMAL1)

MINT_PRICE, MINT_TICK = get_cprice_tick_at_mint(TOKEN0, TOKEN1, FEETIER, MINT_TIME, DECIMAL0, DECIMAL1)
print(MINT_PRICE, 1/MINT_PRICE)
print(MINT_TICK)


# STEP 1 (TODO): CALCULATED BY ALGORITHM, JNP's amounts_relation ?
# THIS CHANGES over time when swaps happens, and we need this to calculate IL
MINT_AMT0 = 252208.7053
MINT_AMT1 = 99.0604

6.0 18.0
 
get_token_id: USDC
{'data': {'tokens': [{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'name': 'USD Coin', 'symbol': 'USDC'}]}}
 
get_token_id: WETH
{'data': {'tokens': [{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 'name': 'Wrapped Ether', 'symbol': 'WETH'}]}}

 get_pool_id for feeTier: 3000
{'data': {'pools': [{'feeTier': '3000', 'id': '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8', 'token0': {'symbol': 'USDC'}, 'token1': {'symbol': 'WETH'}}]}}

    {
      poolHourDatas(first:1,
      where:{ pool: "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8",
      periodStartUnix: 1622304000 }
      ){
        tick
      }
    }
0.00042136376404820986 2373.2463142834135
198600


In [131]:
# STEP 3 (TODO): Generate df_positions
df_positions = get_positions_fixed_strategy(MINT_TIME, BURN_TIME, UPPER, LOWER, DECIMAL0, DECIMAL1)

In [132]:
# Get DF_Merged (accumulated swaps data)
df_merged = pd.read_csv('../data/df_merged_tmp_USDC-WETH-3000-timestamp-1627873200-1620169200.csv')
# df_merged = load_data.get_data(TOKEN0, TOKEN1, FEETIER)

In [133]:
# STEP 4 (TODO): Calculate Fees, IL and PNL at each hour, and append to df_positions
df_positions_fees = calculate_pnl_from_positions(df_merged, df_positions)

In [134]:
# STEP 5 (TODO): also accoutning for minting/burning/swaping costs into PNL

In [146]:
TOKEN0_TO_USD = 1
TOKEN1_TO_USD = 2561.94

totalfees_token0 = df_positions_fees['fees_0'].sum()
totalfees_token1 = df_positions_fees['fees_1'].sum()

fees_token0 = df_positions_fees['fees_0'].sum()
fees_token1 = df_positions_fees['fees_1'].sum()

totalfees_USD = ( fees_token0*TOKEN0_TO_USD )+ (fees_token1 *TOKEN1_TO_USD)

print(fees_token0, fees_token1, totalfees_USD)
revert_fees0 = 37795.612553
revert_fees1 = 16.845536
error_margin_fees0 = (1 - fees_token0/revert_fees0)*100
error_margin_fees1 = (1 - fees_token1/revert_fees1)*100
print('error margin token0 = {} %, token1 = {} %'.format(error_margin_fees0, error_margin_fees1))

35629.55542019289 16.005475229177847 76634.62262883279
error margin token0 = 5.730975069579025 %, token1 = 4.9868450064287195 %


In [ ]:
# # L_you calculation / inRange
# # 4B. JNP: Use 'get_liquidity' function to calculate liquidity as a function of amounts and price range

# # TODO: Dont need this, use JNP code, only need the inRange thing 

# def get_liquidity0(sqrtA,sqrtB,amount0,decimals):
    
#     if (sqrtA > sqrtB):
#           (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
#     liquidity=amount0/((2**96*(sqrtB-sqrtA)/sqrtB/sqrtA)/10**decimals)
#     return liquidity

# def get_liquidity1(sqrtA,sqrtB,amount1,decimals):
    
#     if (sqrtA > sqrtB):
#         (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
#     liquidity=amount1/((sqrtB-sqrtA)/2**96/10**decimals)
#     return liquidity

# def get_liquidity(tick,tickA,tickB,amount0,amount1,decimal0,decimal1):
    
#         sqrt=(1.0001**(tick/2)*(2**96))
#         sqrtA=(1.0001**(tickA/2)*(2**96))
#         sqrtB=(1.0001**(tickB/2)*(2**96))
#         if (sqrtA > sqrtB):
#             (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
#         if sqrt<=sqrtA:
            
#             liquidity0=get_liquidity0(sqrtA,sqrtB,amount0,decimal0)
#             inRange = False
#             return liquidity0, inRange
#         elif sqrt<sqrtB and sqrt>sqrtA:

#             liquidity0=get_liquidity0(sqrt,sqrtB,amount0,decimal0)
#             liquidity1=get_liquidity1(sqrtA,sqrt,amount1,decimal1)
            
#             liquidity=liquidity0 if liquidity0<liquidity1 else liquidity1
#             inRange = True
#             return liquidity, inRange
        
#         else:
#             liquidity1=get_liquidity1(sqrtA,sqrtB,amount1,decimal1)
#             inRange = False
#             return liquidity1, inRange